# Importar librerías

In [214]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import requests
import datetime

# Datos Dólar Informal y Oficial

In [215]:
dfs = []
for tipo_dolar in ['informal', 'oficial']:
    url = f'https://mercados.ambito.com/dolar/{tipo_dolar}/historico-general/01-1-1990/31-12-2050'
    response = requests.get(url)
    data = eval(response.text)
    df = pd.DataFrame(data=data[1:], columns=data[0])
    df['Fecha'] = df['Fecha'].str.replace('\\', '').apply(pd.to_datetime, dayfirst=True)
    df['Venta'] = df['Venta'].str.replace(',', '.').astype(float)
    df = df.set_index('Fecha').sort_index().rename_axis('fecha')
    df = df.Venta.rename(f'venta_{tipo_dolar}')
    df = df.resample('d').mean().ffill()
    dfs.append(df)

df_blue = dfs[0]
df_oficial = dfs[1]

display(df_blue)
display(df_oficial)

C:\Users\leo_a\AppData\Local\Temp\ipykernel_21964\2801354290.py:7: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.

C:\Users\leo_a\AppData\Local\Temp\ipykernel_21964\2801354290.py:7: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



fecha
2002-01-11      1.70
2002-01-12      1.70
2002-01-13      1.70
2002-01-14      1.68
2002-01-15      1.95
               ...  
2023-09-16    730.00
2023-09-17    730.00
2023-09-18    738.00
2023-09-19    740.00
2023-09-20    735.00
Freq: D, Name: venta_informal, Length: 7923, dtype: float64

fecha
2002-04-09      2.85
2002-04-10      2.84
2002-04-11      2.90
2002-04-12      3.05
2002-04-13      3.05
               ...  
2023-09-16    367.00
2023-09-17    367.00
2023-09-18    367.09
2023-09-19    366.64
2023-09-20    367.00
Freq: D, Name: venta_oficial, Length: 7835, dtype: float64

# Datos Inflación

## Inflación Argentina

### Histórica

In [216]:
df_inflacion_arg_hist = pd.read_csv('Argentina_Inflación_Histórica.csv')
df_inflacion_arg_hist['fecha'] = df_inflacion_arg_hist['date'].str.replace('m', '-').apply(pd.to_datetime)
df_inflacion_arg_hist = df_inflacion_arg_hist.set_index('fecha')['mom'].rename('inflacion_arg').loc[:'2016'].dropna()
df_inflacion_arg_hist

fecha
1943-02-01   -0.579
1943-03-01    1.577
1943-04-01    0.663
1943-05-01   -0.869
1943-06-01    0.106
              ...  
2016-08-01    3.118
2016-09-01    1.561
2016-10-01    1.537
2016-11-01    1.785
2016-12-01    1.340
Name: inflacion_arg, Length: 887, dtype: float64

### Actual

In [217]:
df_inflacion_arg_actual = pd.read_csv('https://calculadoradeinflacion.com/argentina.csv')
df_inflacion_arg_actual['fecha'] = df_inflacion_arg_actual['mes'].str.replace('m', '-').apply(pd.to_datetime)
df_inflacion_arg_actual = df_inflacion_arg_actual.set_index('fecha')['inflacion'].rename('inflacion_arg')
df_inflacion_arg_actual

fecha
2017-01-01     1.5859
2017-02-01     2.0670
2017-03-01     2.3140
2017-04-01     2.7330
2017-05-01     1.3760
               ...   
2023-04-01     8.4030
2023-05-01     7.7730
2023-06-01     5.9510
2023-07-01     6.3450
2023-08-01    12.4410
Name: inflacion_arg, Length: 80, dtype: float64

### Esperada

In [218]:
este_mes = datetime.datetime.today().month
for m in [este_mes-1, este_mes-1]:
    m_2_digitos = f'{m:02}'
    for d in [31, 30, 29, 28, 27, 26]:
        try:
            df_inflacion_arg_esperada = pd.read_excel(f'https://www.bcra.gob.ar/Pdfs/PublicacionesEstadisticas/REM23{m_2_digitos}{d}%20Tablas%20web.xlsx').iloc[6:10, 1:5]
            archivo_encontrado = True
            print(f'Archivo del {d} del {m} encontrado.')
            break
        except:
            print(f'Archivo del {d} del {m} no encontrado, continuando...')
            continue
    if archivo_encontrado:
        break
df_inflacion_arg_esperada = df_inflacion_arg_esperada.set_index('Unnamed: 1').rename_axis('fecha')['Unnamed: 3'].rename('inflacion_arg')
df_inflacion_arg_esperada

Archivo del 31 del 8 encontrado.


fecha
2023-09-30          12
2023-10-31    9.111304
2023-11-30    9.382056
2023-12-31        13.2
Name: inflacion_arg, dtype: object

### Total

In [219]:
df_inflacion_arg = pd.concat([df_inflacion_arg_hist, df_inflacion_arg_actual, df_inflacion_arg_esperada]).resample('m').last()*0.01
df_inflacion_arg

fecha
1943-02-28    -0.00579
1943-03-31     0.01577
1943-04-30     0.00663
1943-05-31    -0.00869
1943-06-30     0.00106
                ...   
2023-08-31     0.12441
2023-09-30        0.12
2023-10-31    0.091113
2023-11-30    0.093821
2023-12-31       0.132
Freq: M, Name: inflacion_arg, Length: 971, dtype: object

## Inflación EEUU

In [220]:
!curl -o usd_inflation.csv "https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1318&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=CPIAUCNS&scale=left&cosd=1913-01-01&coed=2023-08-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2023-09-19&revision_date=2050-12-31&nd=1913-01-01"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 22251  100 22251    0     0  20057      0  0:00:01  0:00:01 --:--:-- 20082


In [222]:
df_inflacion_us = pd.read_csv('usd_inflation.csv', index_col=0, parse_dates=True)
df_inflacion_us = df_inflacion_us.rename_axis('fecha')
df_inflacion_us = df_inflacion_us.rename(columns={'CPIAUCNS': 'inflacion_us'})
df_inflacion_us = df_inflacion_us.inflacion_us.pct_change().dropna()
df_inflacion_us = df_inflacion_us.resample('m').last()
df_inflacion_us = df_inflacion_us.reindex(df_inflacion_arg.index).ffill()
df_inflacion_us

fecha
1943-02-28    0.000000
1943-03-31    0.017751
1943-04-30    0.011628
1943-05-31    0.005747
1943-06-30    0.000000
                ...   
2023-08-31    0.004367
2023-09-30    0.004367
2023-10-31    0.004367
2023-11-30    0.004367
2023-12-31    0.004367
Freq: M, Name: inflacion_us, Length: 971, dtype: float64

# Tabla Final

In [292]:
df = pd.concat([df_blue, df_oficial, df_inflacion_arg, df_inflacion_us], axis=1).loc[df_oficial.index[0]:]
df['inflacion_arg'] = df['inflacion_arg'].apply(lambda x:(x+1)**(1/30)).bfill()
df['inflacion_us'] = df['inflacion_us'].apply(lambda x:(x+1)**(1/30)).bfill()
df = df.loc[:datetime.datetime.today()].ffill()
df['informal_ajustado'] = df['venta_informal']*df.inflacion_arg.shift(-1, fill_value=1)[::-1].cumprod()[::-1]/df.inflacion_us.shift(-1, fill_value=1)[::-1].cumprod()[::-1]
df

,venta_informal,venta_oficial,inflacion_arg,inflacion_us,informal_ajustado
fecha,,,,,
2002-04-09,2.80,2.85,1.003300,1.000186,663.043625
2002-04-10,2.80,2.84,1.003300,1.000186,660.985576
2002-04-11,2.84,2.90,1.003300,1.000186,668.347258
2002-04-12,2.98,3.05,1.003300,1.000186,699.117179
2002-04-13,2.98,3.05,1.003300,1.000186,696.947160
...,...,...,...,...,...
2023-09-16,730.00,367.00,1.003785,1.000145,740.683942
2023-09-17,730.00,367.00,1.003785,1.000145,737.998386
2023-09-18,738.00,367.09,1.003785,1.000145,743.380897


In [293]:
px.line(df.reset_index(), x='fecha', y='informal_ajustado', hover_data=['venta_informal'], log_y=True)

In [296]:
from ipywidgets import interact, widgets
import time

# Function to update and display the plot
def update_plot(fecha):
    df_grafico = df.copy()
    fecha = df_grafico.index[fecha]
    ajustador_ar = pd.concat([df_grafico.inflacion_arg.loc[:fecha].shift(-1, fill_value=1)[::-1].cumprod()[::-1],
                   df_grafico.inflacion_arg.loc[fecha:].shift(1, fill_value=1).pipe(lambda x:1/x.cumprod()).iloc[1:]])
    ajustador_us = pd.concat([df_grafico.inflacion_us.loc[:fecha].shift(-1, fill_value=1)[::-1].cumprod()[::-1],
                   df_grafico.inflacion_us.loc[fecha:].shift(1, fill_value=1).pipe(lambda x:1/x.cumprod()).iloc[1:]])

    df_grafico['informal_ajustado'] = df_grafico['venta_informal']*ajustador_ar/ajustador_us
    
    fig = px.line(df_grafico.reset_index(), x='fecha', y='informal_ajustado', hover_data=['venta_informal'], log_y=True, title=f'Precio Dólar Informal a pesos de {fecha.date()}')
    fig.add_vline(x=fecha, line_dash="dash", name="Vertical Line")
    fig.show()

# Create an interactive slider
slider = widgets.IntSlider(value=len(df)-1, min=1, max=len(df), step=30, description='Fecha',
                                 continuous_update=False)

# Define the interaction with the slider
interact(update_plot, fecha=slider)

interactive(children=(IntSlider(value=7834, continuous_update=False, description='Fecha', max=7835, min=1, ste…

<function __main__.update_plot(fecha)>